In [1]:
using Revise
using Pkg; Pkg.activate(".")
using HypergraphModularity

using StatsBase
using Statistics
using Plots
using DataFrames
using SparseArrays
using Clustering

 Activating environment at `~/hypergraph_modularities_code/Project.toml`
┌ Info: Precompiling HypergraphModularity [0c934d27-dd44-49d7-950f-bd4be7819e54]
└ @ Base loading.jl:1260
┌ Warning: Package HypergraphModularity does not have DelimitedFiles in its dependencies:
│ - If you have HypergraphModularity checked out for development and have
│   added DelimitedFiles as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with HypergraphModularity
└ Loading DelimitedFiles into HypergraphModularity from project dependency, future warnings for HypergraphModularity are suppressed.
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



In [42]:
dataset = "email-Enron-full"
kmax_ = 15

H = read_hypergraph_data(dataset,kmax_,2, false)
α = zeros(2*kmax_);

kmax = maximum(keys(H.E))
kmin = minimum(keys(H.E));

n = length(H.D);
D = big(sum(H.D))

4427

In [43]:
D = big(sum(H.D))

ωᵢ = 2.0
ωₒ = 0.6
function ω(p, α)
    num = p[1] == 1 ? ωᵢ : ωₒ
    denom = D^sum(p[2])
    return 500*num / denom
end

Ω̂ = allOrNothingIntensityFunction(ω, maximum(keys(H.E)));

Ẑ_w = SuperNode_PPLouvain(H, Ω̂; α = 0, verbose = false, Z0 = collect(1:n));

println(length(unique(Ẑ_w)))

12


In [44]:
H̄ = projectedGraph(H)

Ω̄ = estimateΩEmpirically(H̄, Ẑ_w; aggregator = p -> [length(p) == 1, sum(p)])

DF = DataFrame()

for i ∈ 1:20
    Ẑ, t = @timed SuperNode_PPLouvain(H̄, Ω̄; α = 0, verbose = false)
    Ω̄ = estimateΩEmpirically(H̄, Ẑ; aggregator = p -> [length(p) == 1, sum(p)])
    
    Q = modularity(H̄, Ẑ, Ω̄; α = 0)
    ℓ = length(unique(Ẑ))
#     ARI = randindex(Z, Ẑ)[1]
    
    df = DataFrame(Q = Q, ℓ = ℓ, method = "dyadic")
    DF = vcat(DF, df)
    
    println(rpad(round(Q), 15, " "), "    ", rpad(ℓ, 10, " "), "    ", rpad(t, 10, " "))
end

-51829.0           18            0.013667617
-51630.0           21            0.013526211
-51581.0           24            0.013985822
-51581.0           23            0.035546895
-51536.0           25            0.014166754
-51581.0           23            0.013497581
-51581.0           24            0.01310611
-51580.0           24            0.014213439
-51581.0           23            0.013276688
-51580.0           24            0.014264732
-51588.0           23            0.034426826
-51584.0           23            0.013396505
-51528.0           24            0.012657007
-51581.0           22            0.013643268
-51536.0           25            0.012549896
-51556.0           23            0.013913386
-51534.0           24            0.013102671
-51579.0           22            0.032382754
-51540.0           24            0.01453839
-51583.0           22            0.013181016


In [45]:
Ω̂ = estimateΩEmpirically(H, Ẑ_w; aggregator = p -> [length(p) == 1, sum(p)])

for i ∈ 1:20
    Ẑ, t = @timed SuperNode_PPLouvain(H, Ω̂; α = 0, verbose = false)
    Ω̂ = estimateΩEmpirically(H, Ẑ; aggregator = p -> [length(p) == 1, sum(p)])
    
    Q = modularity(H, Ẑ, Ω̂; α = 0)
    ℓ = length(unique(Ẑ))
   
    
    df = DataFrame(Q = Q, ℓ = ℓ,  method = "polyadic AON")
    DF = vcat(DF, df)
    
    println(rpad(round(Q), 15, " "), "    ", rpad(ℓ, 10, " "), "    ", rpad(t, 15, " "))
end

-28069.0           14            0.022709803    
-28025.0           15            0.024521547    
-28117.0           15            0.028145216    
-28127.0           14            0.023025176    
-28088.0           15            0.035414966    
-28053.0           16            0.021800327    
-28093.0           16            0.025549311    
-28077.0           16            0.023070474    
-28156.0           16            0.023176631    
-28261.0           19            0.023516995    
-28121.0           17            0.023118071    
-28122.0           17            0.035136139    
-28211.0           19            0.02375035     
-28207.0           19            0.023833868    
-28156.0           17            0.023678493    
-28097.0           17            0.026190843    
-28092.0           17            0.023921212    
-28102.0           17            0.024293674    
-28110.0           17            0.02596995     
-28246.0           14            0.026156314    


In [48]:
n_ = big(n)

function ω(p, α)
    k = big(sum(p))
    return sum(p)/sum((p .* (1:length(p)).^α[k])) / n^(α[kmax+k]*k)
end

# function ω_m(p, α)
#     k = sum(p)
#     kmax = length(α) ÷ 2
#     return (2*p[1]/k)^α[k] / n_^(α[kmax+k]*k)
# end

Ω̂ = partitionIntensityFunction(ω, kmax);
α̂ = vcat(repeat([10.0], kmax), repeat([2.0], kmax));


# Ω̂ = partitionIntensityFunction(ω_m, kmax);
# α̂ = repeat([2.0], 2*kmax)

α̂ = learnParameters(H, Ẑ_w, Ω̂, α̂; max_iters = 50, verbose = false, tol = 1e-4);
Q = modularity(H, Ẑ_w, Ω̂; α = α̂)

-27965.04341865154587550862587419397102482717362729009719287903181776303732784617

In [49]:
for i ∈ 1:20
    Ẑ, t = @timed SuperNodeLouvain(H, Ω̂; α = α̂, verbose = false, scan_order ="random")
    α̂ = learnParameters(H, Ẑ_w, Ω̂, α̂; max_iters = 10, verbose = false, tol = 1e-4)
    
    Q = modularity(H, Ẑ, Ω̂; α = α̂)
    ℓ = length(unique(Ẑ))

    
    df = DataFrame(Q = Q, ℓ = ℓ,  method = "polyadic parameterized")
    DF = vcat(DF, df)
    
    println(rpad(round(Q), 15, " "), "    ", rpad(ℓ, 10, " "), "    ", rpad(t, 15, " ") )
end

-27961.0           13            68.192665672   
-27809.0           12            76.113712758   
-27933.0           13            80.110312093   
-27749.0           12            82.596905925   
-27760.0           12            69.99947189    
-27760.0           13            78.739635056   
-27775.0           13            77.415429139   
-27759.0           13            72.021495514   
-27769.0           12            65.761294166   


InterruptException: InterruptException:

In [50]:
α̂

30-element Array{Float64,1}:
 10.0
  3.764314058623484
  6.097305648267805
  7.453213855516319
  8.319772330295653
  8.28313554036346
  8.306604323029502
  9.971792643563546
  9.039829223803501
  8.069921425982221
 10.213356533224147
 10.181308429823018
 12.025263069787856
  ⋮
  1.2273918538794917
  1.2983993662125064
  1.3412615612621215
  1.3774391157144508
  1.380226392624861
  1.4345604275442574
  1.4528483989154484
  1.4389691067105825
  1.447246532341495
  1.4450268349367807
  2.0
  1.4794101133466422